In [49]:
#!pip install fastai

In [50]:
from flask import Flask, render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
import os
import re

# fixes a dependancy on PosixPath by load_learner
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

import random as rnd
from fastai import *
from fastai.vision.all import *

app = Flask(__name__)
if not os.path.isdir('UploadFolder'): os.mkdir('UploadFolder')
app.config['UPLOAD_FOLDER'] = './static/UploadFolder'

# These are the categories used in the model and the data
DATAPATH = "./static/data/"
CATEGORIES = os.listdir(DATAPATH)
ALLOWED_IMGTYPES = ['png','jpeg','gif','jfif','jpg']
# Every category has at least 600 images
N_MAX = 20

sample = {"sample_class":"sample_hidden"}
 
learn = load_learner('export.pkl')

def format_example(imgpath,cat,prediction):
   return {"image":"/data/"+imgpath,
                           "category":cat,
                           "prediction":prediction[0],
                           "correct_class":"example_"+str(cat==prediction[0])}

def format_sample(imgpath,prediction):
   return {"image":"/UploadFolder/"+imgpath,
            "prediction":prediction[0],
            "sample_class":"sample_visable"}

def get_examples():
   examples = []
   for cat in CATEGORIES:
      n = rnd.choice(range(N_MAX))
      imgpath = cat+"/"+cat+str(n)+".png"
      prediction = learn.predict(DATAPATH+imgpath)

      examples.append(format_example(imgpath,cat,prediction))
   
   return examples


   
    


@app.route('/')
def home():
   global sample
   print(sample)
   examples = get_examples()
   return render_template('home.html',examples=examples,sample=sample)

@app.route("/testImage_post", methods=['post'])
def testImage_post():




   # for img in os.listdir(app.config['UPLOAD_FOLDER']):
   #    os.remove(os.path.join(app.config['UPLOAD_FOLDER'],img))
   global sample
   # examples are manualy set for now, straight from the model? or select some to test and test them and than compare to name using regex?

   image = request.files['file']
   #regex function to see if uploaded file was an image

   imgpath = secure_filename(image.filename)

   print("Image path:" + imgpath)
   
   

   if re.sub('.*\\.','',imgpath) not in ALLOWED_IMGTYPES:
      return redirect(url_for('home'))

   image.save(os.path.join(app.config['UPLOAD_FOLDER'], imgpath))

   #test the posted image with the model
   prediction = learn.predict(os.path.join(app.config['UPLOAD_FOLDER'], imgpath))

   sample = format_sample(imgpath,prediction)



   return redirect(url_for('home'))



if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Image path:gothic10.png


127.0.0.1 - - [11/Dec/2021 15:16:31] "POST /testImage_post HTTP/1.1" 302 -


{'image': '/UploadFolder/gothic10.png', 'prediction': 'gothic', 'sample_class': 'sample_visable'}


...

127.0.0.1 - - [11/Dec/2021 15:19:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2021 15:19:18] "GET /static//data/gothic/gothic13.png HTTP/1.1" 308 -
127.0.0.1 - - [11/Dec/2021 15:19:18] "GET /static//data/tudor/tudor18.png HTTP/1.1" 308 -
127.0.0.1 - - [11/Dec/2021 15:19:18] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2021 15:19:18] "GET /static/data/art%20deco/art%20deco5.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2021 15:19:18] "GET /static/data/baroque/baroque15.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2021 15:19:18] "GET /static/data/roman/roman15.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2021 15:19:18] "GET /static/data/gothic/gothic13.png HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2021 15:19:18] "GET /static//UploadFolder/idVoorkant.jpg HTTP/1.1" 308 -
127.0.0.1 - - [11/Dec/2021 15:19:18] "GET /static/data/tudor/tudor18.png HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2021 15:19:19] "GET /static/UploadFolder/idVoorkant.jpg HTTP/1.1" 200 -


Image path:4k-resolution-wallpapers-Ultra-HD.jpg


127.0.0.1 - - [11/Dec/2021 15:19:31] "POST /testImage_post HTTP/1.1" 302 -


{'image': '/UploadFolder/4k-resolution-wallpapers-Ultra-HD.jpg', 'prediction': 'gothic', 'sample_class': 'sample_visable'}


127.0.0.1 - - [11/Dec/2021 15:19:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2021 15:19:31] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2021 15:19:32] "GET /static/data/baroque/baroque9.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2021 15:19:32] "GET /static/data/art%20deco/art%20deco6.png HTTP/1.1" 200 -
127.0.0.1 - - [11/Dec/2021 15:19:32] "GET /static/data/gothic/gothic3.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2021 15:19:32] "GET /static/data/roman/roman7.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2021 15:19:32] "GET /static//UploadFolder/4k-resolution-wallpapers-Ultra-HD.jpg HTTP/1.1" 308 -
127.0.0.1 - - [11/Dec/2021 15:19:32] "GET /static/data/tudor/tudor15.png HTTP/1.1" 304 -
127.0.0.1 - - [11/Dec/2021 15:19:32] "GET /static/UploadFolder/4k-resolution-wallpapers-Ultra-HD.jpg HTTP/1.1" 200 -
